In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install nemo_toolkit[all]

In [ ]:
pip install nemo_toolkit[asr]

In [ ]:
pip install nemo_asr

In [6]:
pip install pysoundfile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
manifest_train = '/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/manifest_test.json'
manifest_test = '/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/manifest_test.json'
yaml_target = '/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject'
checkpoint_path = '/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/asr_checkpoint.ckpt'

In [19]:
import yaml

# Define the ASR model parameters
params = {'name': 'QuartzNet15x5', 
      'sample_rate': 16000, 
      'repeat': 1, 
      'dropout': 0.0, 
      'separable': True, 
      'labels': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"], 
      'model': {'train_ds': {'manifest_filepath': manifest_train, 'sample_rate': 16000, 
                'labels': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"], 
                'batch_size': 2, 'trim_silence': True, 'max_duration': 16.7, 'shuffle': True, 'num_workers': 8, 
                'pin_memory': True, 'is_tarred': False, 'tarred_audio_filepaths': None, 'shuffle_n': 2048, 
                'bucketing_strategy': 'synced_randomized', 'bucketing_batch_size': None}, 
                'validation_ds': {'manifest_filepath': manifest_test, 'sample_rate': 16000, 
                'labels': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"], 
                'batch_size': 2, 'shuffle': False, 'num_workers': 8, 'pin_memory': True}, 
                'preprocessor': {'_target_': 'nemo.collections.asr.modules.AudioToMelSpectrogramPreprocessor', 
                'normalize': 'per_feature', 'window_size': 0.02, 'sample_rate': 16000, 'window_stride': 0.01, 'window': 'hann', 'features': 64, 
                'n_fft': 512, 'frame_splicing': 1, 'dither': 1e-05, 'stft_conv': False}, 
                'spec_augment': {'_target_': 'nemo.collections.asr.modules.SpectrogramAugmentation', 'rect_freq': 50, 'rect_masks': 5, 'rect_time': 120}, 
                'encoder': {'_target_': 'nemo.collections.asr.modules.ConvASREncoder', 'feat_in': 64, 'activation': 'relu', 'conv_mask': True, 
                'jasper': [{'filters': 128, 'repeat': 1, 'kernel': [11], 'stride': [1], 'dilation': [1], 'dropout': 0.0, 'residual': True, 'separable': True,
                'se': True, 'se_context_size': -1}, {'filters': 256, 'repeat': 1, 'kernel': [13], 'stride': [1], 'dilation': [1], 'dropout': 0.0, 'residual': True, 'separable': True, 'se': True, 'se_context_size': -1}, 
                {'filters': 256, 'repeat': 1, 'kernel': [15], 'stride': [1], 'dilation': [1], 'dropout': 0.0, 'residual': True, 'separable': True, 'se': True, 'se_context_size': -1}, 
                {'filters': 256, 'repeat': 1, 'kernel': [17], 'stride': [1], 'dilation': [1], 'dropout': 0.0, 'residual': True, 'separable': True, 'se': True, 'se_context_size': -1}, 
                {'filters': 256, 'repeat': 1, 'kernel': [19], 'stride': [1], 'dilation': [1], 'dropout': 0.0, 'residual': True, 'separable': True, 'se': True, 'se_context_size': -1}, 
                {'filters': 256, 'repeat': 1, 'kernel': [21], 'stride': [1], 'dilation': [1], 'dropout': 0.0, 'residual': False, 'separable': True, 'se': True, 'se_context_size': -1}, 
                {'filters': 1024, 'repeat': 1, 'kernel': [1], 'stride': [1], 'dilation': [1], 'dropout': 0.0, 'residual': False, 'separable': True, 'se': True, 'se_context_size': -1}]}, 
                'decoder': {'_target_': 'nemo.collections.asr.modules.ConvASRDecoder', 'feat_in': 1024, 'num_classes': 28, 
                'vocabulary': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"]}, 
                'optim': {'name': 'novograd', 'lr': 0.01, 'betas': [0.8, 0.5], 'weight_decay': 0.001, 
                'sched': {'name': 'CosineAnnealing', 'monitor': 'val_loss', 'reduce_on_plateau': False, 'warmup_steps': None, 'warmup_ratio': None, 'min_lr': 0.0, 'last_epoch': -1}}}
                , 'trainer': {'devices': 1, 'max_epochs': 5, 'max_steps': -1, 'num_nodes': 1, 'accelerator': 'gpu', 'strategy': 'ddp', 'accumulate_grad_batches': 1, 
                              'enable_checkpointing': False, 'logger': False, 'log_every_n_steps': 1, 'val_check_interval': 1.0, 'benchmark': False}, 
      'exp_manager': {'exp_dir': None, 'name': 'QuartzNet15x5', 'create_tensorboard_logger': True, 'create_checkpoint_callback': True}}


# Write the parameters to a YAML file
with open(yaml_target+'/asr_config.yaml', 'w') as f:
    yaml.dump(params, f)


In [9]:
import pytorch_lightning as pl
trainer = pl.Trainer(devices=1, accelerator='gpu', max_epochs=50)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [20]:
import yaml

# Open the YAML file
with open(yaml_target+'/asr_config.yaml', 'r') as f:
    params = yaml.safe_load(f)
print(params)

{'dropout': 0.0, 'exp_manager': {'create_checkpoint_callback': True, 'create_tensorboard_logger': True, 'exp_dir': None, 'name': 'QuartzNet15x5'}, 'labels': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"], 'model': {'decoder': {'_target_': 'nemo.collections.asr.modules.ConvASRDecoder', 'feat_in': 1024, 'num_classes': 28, 'vocabulary': [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', "'"]}, 'encoder': {'_target_': 'nemo.collections.asr.modules.ConvASREncoder', 'activation': 'relu', 'conv_mask': True, 'feat_in': 64, 'jasper': [{'dilation': [1], 'dropout': 0.0, 'filters': 128, 'kernel': [11], 'repeat': 1, 'residual': True, 'se': True, 'se_context_size': -1, 'separable': True, 'stride': [1]}, {'dilation': [1], 'dropout': 0.0, 'filters': 256, 'kernel': [13], 'repeat': 1, 'residual': True, 'se': True, 'se_context_s

In [12]:
from omegaconf import DictConfig
import nemo.collections.asr as nemo_asr

first_asr_model = nemo_asr.models.EncDecCTCModel(cfg=DictConfig(params['model']), trainer=trainer)


[NeMo W 2023-04-03 14:13:55 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2023-04-03 14:14:00 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.


[NeMo I 2023-04-03 14:14:00 audio_to_text_dataset:45] Model level config does not contain `sample_rate`, please explicitly provide `sample_rate` to the dataloaders.
[NeMo I 2023-04-03 14:14:00 audio_to_text_dataset:45] Model level config does not contain `labels`, please explicitly provide `labels` to the dataloaders.
[NeMo I 2023-04-03 14:14:01 collections:193] Dataset loaded with 1128 files totalling 1.84 hours
[NeMo I 2023-04-03 14:14:01 collections:194] 82 files were filtered totalling 0.63 hours


[NeMo W 2023-04-03 14:14:01 nemo_logging:349] /usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      warnings.warn(_create_warning_msg(
    


[NeMo I 2023-04-03 14:14:01 audio_to_text_dataset:45] Model level config does not contain `sample_rate`, please explicitly provide `sample_rate` to the dataloaders.
[NeMo I 2023-04-03 14:14:01 audio_to_text_dataset:45] Model level config does not contain `labels`, please explicitly provide `labels` to the dataloaders.
[NeMo I 2023-04-03 14:14:01 collections:193] Dataset loaded with 1210 files totalling 2.47 hours
[NeMo I 2023-04-03 14:14:01 collections:194] 0 files were filtered totalling 0.00 hours
[NeMo I 2023-04-03 14:14:01 features:286] PADDING: 16


In [13]:
# Start training!!!
trainer.fit(first_asr_model)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[NeMo I 2023-04-03 14:15:21 modelPT:720] Optimizer config = Novograd (
    Parameter Group 0
        amsgrad: False
        betas: [0.8, 0.5]
        eps: 1e-08
        grad_averaging: False
        lr: 0.01
        weight_decay: 0.001
    )
[NeMo I 2023-04-03 14:15:21 lr_scheduler:910] Scheduler "<nemo.core.optim.lr_scheduler.CosineAnnealing object at 0x7f0e044533a0>" 
    will be used during training (effective maximum steps = 1800) - 
    Parameters : 
    (last_epoch: -1
    min_lr: 0.0
    warmup_ratio: null
    warmup_steps: null
    max_steps: 1800
    )


INFO:pytorch_lightning.callbacks.model_summary:
  | Name              | Type                              | Params
------------------------------------------------------------------------
0 | preprocessor      | AudioToMelSpectrogramPreprocessor | 0     
1 | encoder           | ConvASREncoder                    | 1.2 M 
2 | decoder           | ConvASRDecoder                    | 29.7 K
3 | loss              | CTCLoss                           | 0     
4 | spec_augmentation | SpectrogramAugmentation           | 0     
5 | _wer              | WER                               | 0     
------------------------------------------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.836     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

[NeMo W 2023-04-03 14:15:26 nemo_logging:349] /usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
      warnings.warn(_create_warning_msg(
    
[NeMo W 2023-04-03 14:15:32 nemo_logging:349] /usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/trainer.py:1600: PossibleUserWarning: The number of training batches (36) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
      rank_zero_warn(
    


...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.


In [17]:
first_asr_model.save_to(checkpoint_path)

In [22]:
# Bigger batch-size = bigger throughput
params['model']['validation_ds']['batch_size'] = 2

# Setup the test data loader and make sure the model is on GPU
first_asr_model.setup_test_data(test_data_config=params['model']['validation_ds'])
first_asr_model.cuda()
first_asr_model.eval()

# We will be computing Word Error Rate (WER) metric between our hypothesis and predictions.
# WER is computed as numerator/denominator.
# We'll gather all the test batches' numerators and denominators.
wer_nums = []
wer_denoms = []

# Loop over all test batches.
# Iterating over the model's `test_dataloader` will give us:
# (audio_signal, audio_signal_length, transcript_tokens, transcript_length)
# See the AudioToCharDataset for more details.
for test_batch in first_asr_model.test_dataloader():
        test_batch = [x.cuda() for x in test_batch]
        targets = test_batch[2]
        targets_lengths = test_batch[3]        
        log_probs, encoded_len, greedy_predictions = first_asr_model(
            input_signal=test_batch[0], input_signal_length=test_batch[1]
        )
        # Notice the model has a helper object to compute WER
        first_asr_model._wer.update(greedy_predictions, targets, targets_lengths)
        _, wer_num, wer_denom = first_asr_model._wer.compute()
        first_asr_model._wer.reset()
        wer_nums.append(wer_num.detach().cpu().numpy())
        wer_denoms.append(wer_denom.detach().cpu().numpy())

        # Release tensors from GPU memory
        del test_batch, log_probs, targets, targets_lengths, encoded_len, greedy_predictions

# We need to sum all numerators and denominators first. Then divide.
print(f"WER = {sum(wer_nums)/sum(wer_denoms)}")

[NeMo I 2023-04-03 15:08:38 audio_to_text_dataset:45] Model level config does not contain `sample_rate`, please explicitly provide `sample_rate` to the dataloaders.
[NeMo I 2023-04-03 15:08:38 audio_to_text_dataset:45] Model level config does not contain `labels`, please explicitly provide `labels` to the dataloaders.
[NeMo I 2023-04-03 15:08:38 collections:193] Dataset loaded with 1210 files totalling 2.47 hours
[NeMo I 2023-04-03 15:08:38 collections:194] 0 files were filtered totalling 0.00 hours
WER = 0.852633424913778
